In [ ]:
from os import getenv
from connect_db import connect_db
con = connect_db(
    username=getenv("ORACLE_USERNAME"),
    password=getenv("ORACLE_PASSWORD"),
    host=getenv("ORACLE_HOST"),
    port=int(getenv("ORACLE_PORT"))
)
cur = con.cursor()

In [ ]:
FIRST_CLASS_PERCENTAGE = 0.15
BUSINESS_CLASS_PERCENTAGE = 0.30
ECONOMY_CLASS_PERCENTAGE = 0.55

BUSINESS_CLASS_SEATS_PER_ROW = 4
ECONOMY_CLASS_SEAT_PER_ROW = 6
ALPHABETS = "ABCDEF"

In [ ]:
FIRST_CLASS_CUTOFF = lambda n_seats: round(n_seats * FIRST_CLASS_PERCENTAGE)
BUSINESS_CLASS_CUTOFF = lambda n_seats: round(n_seats * BUSINESS_CLASS_PERCENTAGE)

def get_location_id(n: int, total_number_of_seats: int, seat_class: str) -> str:
    match seat_class:
        case "FIRS":
            return f"S{n}"
        case "BUSI":
            n = n - FIRST_CLASS_CUTOFF(total_number_of_seats)
            row, col = divmod(n, BUSINESS_CLASS_SEATS_PER_ROW)
            return f"B{row + 1}{ALPHABETS[col]}"
        case "ECON":
            n = n - BUSINESS_CLASS_CUTOFF(total_number_of_seats)
            row, col = divmod(n, ECONOMY_CLASS_SEAT_PER_ROW)
            return f"{row + 1}{ALPHABETS[col]}"
        case _:
            print("What")
            return ""

def get_class_id(n: int, total_number_of_seats: int) -> str:
    if n < FIRST_CLASS_CUTOFF(total_number_of_seats):
        return "FIRS"
    
    if n < BUSINESS_CLASS_CUTOFF(total_number_of_seats):
        return "BUSI"
    
    return "ECON"

In [ ]:
stmt = """
SELECT AIRCRAFT_ID, NUMBER_OF_SEAT 
FROM AIRCRAFT
    JOIN AIRCRAFT_MODEL ON AIRCRAFT.AIRCRAFT_MODEL_ID = AIRCRAFT_MODEL.AIRCRAFT_MODEL_ID
"""
query = "    INTO SEAT (AIRCRAFT_ID, SEAT_CLASS_ID, LOCATION) VALUES ('{}', '{}', '{}')"
for row in cur.execute(stmt):
    aircraft_id = row[0]
    number_of_seat = row[1]
    
    for i in range(1, number_of_seat + 1):
        seat_class_id = get_class_id(i, number_of_seat)
        location = get_location_id(i, number_of_seat, seat_class_id)
        print(query.format(aircraft_id, seat_class_id, location))
    
    

In [ ]:
con.close()